# CSV file to NP classification
## Change the parameter here:

In [54]:
csv = '/Users/shunyang.wang/project/SIRIUS/SIRIUS_utils/benchmark/pepsearch_tsv/IND.csv'# input csv
smi = 'smiles'# column name of smiles

In [55]:
import pandas as pd
import grequests
import urllib.parse
from tqdm import tqdm
import collections

In [64]:
SERVER_URL = "http://192.168.128.240:6541/"

df = pd.read_csv(csv)

all_urls = []

for entry in tqdm(df.to_dict(orient="records")):
    smiles = str(entry[smi])
    if len(smiles) > 5:
        request_url = "{}/classify?smiles={}".format(SERVER_URL, urllib.parse.quote(smiles))
        all_urls.append(request_url)

# Lets actually do the query and measure the speed
rs = (grequests.get(u) for u in all_urls)
responses = grequests.map(rs, size=20)


100%|█████████████████████████████████████████████████████████████████████████████| 115/115 [00:00<00:00, 211740.54it/s]


In [58]:
# get output dataframe
class_list = []
superclass = []
pathway = []
gly = []
for item in responses:
    try:
        tmp = item.json()
    except:
        tmp = {'class_results':'nan', 'superclass_results':'nan',
                  'pathway_results':'nan', 'isglycoside':'nan'}
    class_list.append(tmp['class_results'])
    superclass.append(tmp['superclass_results'])
    pathway.append(tmp['pathway_results'])
    gly.append(tmp['isglycoside'])
    
classfy = pd.DataFrame({'class_results':class_list, 'superclass_results':superclass,
                  'pathway_results':pathway, 'isglycoside':gly})

In [59]:
df2 = pd.concat([df,classfy],axis=1)

In [60]:
df2.to_csv(csv.replace('.csv','npclass.csv'),index=False)

In [62]:
df2

,Unnamed: 0.1,Unnamed: 0,Num,Unknown,InChIKey,Precursor m/z,Rank,Library,Id,Mass,...,NIST r.n.,Num.Comp,mode,column,inchi,smiles,class_results,superclass_results,pathway_results,isglycoside
0,0,116,1201,IND_HH_F_PFP_M4_P_CIDHCD_9092022_IDENTIFICATIO...,IIRDTKBZINWQAW-UHFFFAOYSA-N,283.1750,1.0,hr_msms_nist,79551.0,282.168,...,1186838.0,9.0,P,PFP,InChI=1S/C12H26O7/c13-1-3-15-5-7-17-9-11-19-12...,OCCOCCOCCOCCOCCOCCO,[Monoacylglycerols],[Glycerolipids],[Fatty acids],False
1,1,190,1696,IND_HH_F_PFP_M4_P_CIDHCD_9092022_IDENTIFICATIO...,RSWBWHPZXKLUEX-VOTSOKGWSA-N,145.0652,1.0,hr_msms_nist,308552.0,162.068,...,1525554.0,255.0,P,PFP,InChI=1S/C10H10O2/c1-8-4-2-3-5-9(8)6-7-10(11)1...,Cc1ccccc1/C=C/C(=O)O,[Cinnamic acids and derivatives],[Phenylpropanoids (C6-C3)],[Shikimates and Phenylpropanoids],False
2,2,201,1849,IND_HH_F_PFP_M4_P_CIDHCD_9092022_IDENTIFICATIO...,XEVQXKKKAVVSMW-WRWORJQWSA-N,197.1172,1.0,hr_msms_nist,722589.0,196.110,...,3101076.0,57.0,P,PFP,InChI=1S/C11H16O3/c1-10(2)5-7(12)6-11(3)8(10)4...,CC1(C)C[C@H](O)C[C@@]2(C)OC(=O)C=C12,[Apocarotenoids (β-)],[Apocarotenoids],[Terpenoids],False
3,3,202,1851,IND_HH_F_PFP_M4_P_CIDHCD_9092022_IDENTIFICATIO...,IJGMVUXEZUEDJR-RTWAVKEYSA-N,179.1060,1.0,hr_msms_nist,607212.0,234.162,...,1948004.0,222.0,P,PFP,InChI=1S/C15H22O2/c1-9-11-4-5-12(13(16)17)15(1...,C=C1[C@H]2CC[C@H](C(=O)O)[C@@]23CC[C@H](C3)C1(C)C,[Zizaane sesquiterpenoids],[Sesquiterpenoids],[Terpenoids],False
4,4,203,1852,IND_HH_F_PFP_M4_P_CIDHCD_9092022_IDENTIFICATIO...,LXOFYPKXCSULTL-UHFFFAOYSA-N,135.1162,1.0,hr_msms_nist,110830.0,226.193,...,1236765.0,1003.0,P,PFP,"InChI=1S/C14H26O2/c1-11(2)9-13(5,15)7-8-14(6,1...",CC(C)CC(C)(O)C#CC(C)(O)CC(C)C,[],[],[Terpenoids],False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,110,239,2293,IND_HH_F_PFP_M2_P_CIDHCD_9092022_IDENTIFICATIO...,AUJXJFHANFIVKH-GQCTYLIASA-N,177.0548,1.0,hr_msms_nist,306285.0,208.074,...,1522011.0,705.0,P,PFP,InChI=1S/C11H12O4/c1-14-10-7-8(3-5-9(10)12)4-6...,COC(=O)/C=C/c1ccc(O)c(OC)c1,[Cinnamic acids and derivatives],[Phenylpropanoids (C6-C3)],[Shikimates and Phenylpropanoids],False
111,111,240,2294,IND_HH_F_PFP_M2_P_CIDHCD_9092022_IDENTIFICATIO...,QAIPRVGONGVQAS-DUXPYHPUSA-N,145.0286,1.0,hr_msms_nist,187819.0,180.042,...,1350786.0,61.0,P,PFP,InChI=1S/C9H8O4/c10-7-3-1-6(5-8(7)11)2-4-9(12)...,O=C(O)/C=C/c1ccc(O)c(O)c1,[Cinnamic acids and derivatives],[Phenylpropanoids (C6-C3)],[Shikimates and Phenylpropanoids],False
112,112,264,2602,IND_HH_F_PFP_M2_P_CIDHCD_9092022_IDENTIFICATIO...,DOUMFZQKYFQNTF-WUTVXBCWSA-N,163.0385,1.0,hr_msms_nist,615285.0,360.085,...,1958528.0,609.0,P,PFP,InChI=1S/C18H16O8/c19-12-4-1-10(7-14(12)21)3-6...,O=C(/C=C/c1ccc(O)c(O)c1)O[C@H](Cc1ccc(O)c(O)c1...,[Cinnamic acids and derivatives],[Phenylpropanoids (C6-C3)],[Shikimates and Phenylpropanoids],False
113,113,291,2767,IND_HH_F_PFP_M2_P_CIDHCD_9092022_IDENTIFICATIO...,FETSKTIMHFKZNF-AATRIKPKSA-N,175.1481,1.0,hr_msms_nist,915029.0,192.151,...,3347705.0,57.0,P,PFP,InChI=1S/C13H20O/c1-9-7-10(2)13(11(3)8-9)6-5-1...,CC(=O)/C=C/C1C(C)C=C(C)CC1C,[],[],[Terpenoids],False
